In [1]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import sys
import pandas as pd
import requests
from api_keys import AV_api

In [2]:
# define sectors
sectors = {
    'Semiconductors': ['QCOM', 'TXN', 'ADI', 'MU', 'ARM'],
    'Software': ['CRM', 'SAP', 'INTU', 'NOW', 'ZI'],
    'Consumer': ['AAPL', 'MSFT', 'SONY', 'LPL', 'SONO'],
    'Infotech': ['ACN', 'IBM', 'FI', 'INFY', 'CTSH']
}
sectors = pd.DataFrame(sectors)


In [3]:
all_data = []                                                                   # create an empty list to append to

start_date = '2018-01-01'                                                       # create start and end dates

end_date = '2023-12-04'

for industry, companies in sectors.items():                                       # iterate over each industry
    for ticker in companies:                                                    # iterate over each company in industry
        try:
            print (f'Gathering data for {industry}: {ticker}')
            
            ts = TimeSeries(key='1CSXU4M8SUPKSPG', output_format='pandas')     # make api call
            data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')
            data_df = pd.DataFrame(data)

            data_df = data_df.sort_index()                                      # filter from start to end dates
            filtered_data = pd.DataFrame(data_df.loc[start_date:end_date])

            #Ticker, sector, Industry, Open, High, Low, Close, Volume
            filtered_data["Ticker"] = ticker                                    # add ticker column
            filtered_data["Industry"] = industry
            filtered_data["Sector"] = 'Technology'
            all_data.append(filtered_data)
        except:
            print(f'Company {ticker} not found')

Gathering data for Semiconductors: QCOM
Gathering data for Semiconductors: TXN
Gathering data for Semiconductors: ADI
Gathering data for Semiconductors: MU
Gathering data for Semiconductors: ARM
Gathering data for Software: CRM
Gathering data for Software: SAP
Gathering data for Software: INTU
Gathering data for Software: NOW
Gathering data for Software: ZI
Gathering data for Consumer: AAPL
Gathering data for Consumer: MSFT
Gathering data for Consumer: SONY
Gathering data for Consumer: LPL
Gathering data for Consumer: SONO
Gathering data for Infotech: ACN
Gathering data for Infotech: IBM
Gathering data for Infotech: FI
Gathering data for Infotech: INFY
Gathering data for Infotech: CTSH


In [4]:
final_data = pd.concat(all_data)                        # Convert to dataframe


In [5]:
all_data

[            1. open  2. high   3. low  4. close   5. volume Ticker  \
 date                                                                 
 2018-01-02    64.38   65.330   64.170     65.20   6299841.0   QCOM   
 2018-01-03    65.22   66.060   64.690     65.94   6721391.0   QCOM   
 2018-01-04    66.18   66.530   65.910     66.03   5035323.0   QCOM   
 2018-01-05    66.04   66.600   65.500     66.47   5345423.0   QCOM   
 2018-01-08    66.25   66.530   65.940     66.27   4253127.0   QCOM   
 ...             ...      ...      ...       ...         ...    ...   
 2023-11-28   127.20  127.305  125.670    126.59   8817854.0   QCOM   
 2023-11-29   129.24  130.600  127.620    127.91  10608317.0   QCOM   
 2023-11-30   129.27  129.400  127.485    129.05  11731774.0   QCOM   
 2023-12-01   129.06  130.270  128.510    129.67   8866491.0   QCOM   
 2023-12-04   129.33  130.050  128.030    129.95   7485048.0   QCOM   
 
                   Industry      Sector  
 date                            

In [6]:
final_data = final_data.reset_index()
final_data.to_csv('raw_data_update.csv', index=True)        # Save to csv

In [9]:
tech_data = pd.read_csv('raw_data_update.csv')
tech_data = tech_data.rename(columns={'date': 'Date',
                                      '1. open': 'Open', 
                                      '2. high' : 'High', 
                                      '3. low' : 'Low',
                                      '4. close': 'Close',
                                      '5. volume': 'Volume',
                                      'Companies': 'sector'})
tech_data['sector'] = 'Technology'
tech_data = tech_data[['Date','Ticker', 'Sector', 'Industry', 'Open', 'High', 'Low', 'Close', 'Volume']]
# tech_data.to_csv('tech_data.csv', index = False)
tech_data['Date'] = pd.to_datetime(tech_data['Date'])
tech_data.to_csv('tech_data_update.csv')

In [10]:
pd.read_csv('tech_data_update.csv').drop(columns={'Unnamed: 0'})

,Date,Ticker,Sector,Industry,Open,High,Low,Close,Volume
0,2018-01-02,QCOM,Technology,Semiconductors,64.38,65.33,64.170,65.20,6299841.0
1,2018-01-03,QCOM,Technology,Semiconductors,65.22,66.06,64.690,65.94,6721391.0
2,2018-01-04,QCOM,Technology,Semiconductors,66.18,66.53,65.910,66.03,5035323.0
3,2018-01-05,QCOM,Technology,Semiconductors,66.04,66.60,65.500,66.47,5345423.0
4,2018-01-08,QCOM,Technology,Semiconductors,66.25,66.53,65.940,66.27,4253127.0
...,...,...,...,...,...,...,...,...,...
27625,2023-11-28,CTSH,Technology,Infotech,70.70,71.52,70.700,71.30,2607377.0
27626,2023-11-29,CTSH,Technology,Infotech,71.60,71.80,69.895,70.12,3521683.0
27627,2023-11-30,CTSH,Technology,Infotech,70.39,70.52,69.800,70.38,8254394.0
27628,2023-12-01,CTSH,Technology,Infotech,70.23,70.87,70.080,70.82,2672566.0
